In [1]:
%cd ..

/Users/koloss/Desktop/CS224N/final_project/RobustQA


In [12]:
from transformers import DistilBertTokenizerFast
import util

In [38]:
import pandas as pd
import numpy as np

In [ ]:

def get_dataset(args, datasets, data_dir, tokenizer, split_name, debug=-1):
    datasets = datasets.split(',')
    dataset_dict = None
    dataset_name = ''
    label = 0
    for dataset in datasets:
        dataset_name += f'_{dataset}'
        dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
        dataset_dict_curr['label'] = label
        dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
        label += 1
    data_encodings = read_and_process(args, tokenizer, dataset_dict, data_dir, dataset_name, split_name)
    return util.QADataset(data_encodings, train=(split_name == 'train')), dataset_dict, label


In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [166]:
datasets = 'duorc,race'
data_dir = 'datasets/oodomain_train'

In [167]:
datasets = datasets.split(',')
dataset_dict = None
dataset_name = ''
label = 0
for dataset in datasets:
    dataset_name += f'_{dataset}'
    dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
    dataset_dict_curr['label'] = label
    dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
    label += 1
#data_encodings = read_and_process(args, tokenizer, dataset_dict, data_dir, dataset_name, split_name)



In [168]:
dataset_dict.keys()

dict_keys(['question', 'context', 'id', 'answer', 'label'])

In [169]:
changes=dict(zip([" game ", " set "], [" lame ", " bet "]))

df = pd.DataFrame({x: dataset_dict[x] for x in dataset_dict if x not in ['label']})
df['start_char'] = df.answer.apply(lambda x : x['answer_start'][0])
df['end_char'] = df['start_char']+ df.answer.apply(lambda x : len(x['text'][0]))
df['final_answer'] = [A[B:C] for A, B, C in zip(df.context, df['start_char'],df['end_char'])]
df['context'] = df.context.str.strip().replace(changes,regex=True)

##df['new_context'] = df.context.str.strip().replace(changes,regex=True)
##df['new_answer'] = [A[B:C] for A, B, C in zip(df['new_context'], df['start_char'],df['end_char'])]
new_dataset_dict = df[[i for i in dataset_dict.keys() if i!= 'label']].to_dict()
new_dataset_dict ['label'] = dataset_dict['label']

In [156]:
#df[[i for i in dataset_dict.keys() if i!= 'label']].to_dict()

In [172]:
df.head()

,question,context,id,answer,start_char,end_char,final_answer
0,What was Jill's mother's face burned by?,"New Orleans, Louisiana, 1927. An enraged posse...",d94a42693350473581ff79dc91c91e04,"{'answer_start': [2476], 'text': ['acid']}",2476,2480,acid
1,What year did a lynch mod muder someone who th...,"New Orleans, Louisiana, 1927. An enraged posse...",f41dbe24bed44870a8ad36c87dda59a2,"{'answer_start': [26], 'text': ['1927']}",26,30,1927
2,Whose corpse is in the morgue?,"New Orleans, Louisiana, 1927. An enraged posse...",34f8093a16b64c8097bdaa03cccdef37,"{'answer_start': [2873], 'text': ['Mary-Anne']}",2873,2882,Mary-Anne
3,What is the name of the blind girl?,"New Orleans, Louisiana, 1927. An enraged posse...",b6a13cac6289435697e8ff98d55854a9,"{'answer_start': [3659], 'text': ['Emily']}",3659,3664,Emily
4,Whose corpse does Liza see?,"New Orleans, Louisiana, 1927. An enraged posse...",52ffe790d24a41669c08a240c1d45114,"{'answer_start': [2348], 'text': ['Jill']}",2348,2352,Jill


## SR

In [173]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet 

[nltk_data] Downloading package wordnet to /Users/koloss/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /Users/koloss/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [193]:

import random
from random import shuffle

In [194]:
#stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '']

#cleaning up text
import re
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line


In [195]:

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

def eda(sentence, alpha_sr=0.1, alpha_ri=0, alpha_rs=0, p_rd=0, num_aug=9):
	
	sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not '']
	num_words = len(words)
	
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	#trim so that we have the desired number of augmented sentences
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	#append the original sentence
	augmented_sentences.append(sentence)

	return augmented_sentences

In [196]:
tst = df.loc[0,'question']
tst

"What was Jill's mother's face burned by?"

In [197]:
eda(tst)

['what was jills mothers face sting by',
 'what was jills mothers face glow by',
 'what was jills mothers face up burned by',
 'what was jills mothers face burned by ']